In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns
import matplotlib.pyplot as plt
import arima
import datetime
from sklearn.preprocessing import Imputer
%matplotlib inline

import sys
sys.path.append('../')
from pipeline import *
from onehot import *
from util import *


In [ ]:
features = pd.read_csv('raw_data/features.csv')
features.head(5)

In [ ]:
features[np.logical_not(np.isnan(features.MarkDown3))].head()

In [ ]:
stores = pd.read_csv('raw_data/stores.csv')
stores.head(5)

In [ ]:
train_data = pd.read_csv('raw_data/train.csv')
test_data = pd.read_csv('raw_data/test.csv')

use lr

In [ ]:
# full_train=train_data.merge(stores,left_on='Store',right_on='Store').merge(features,left_on=('Store','Date'),right_on=('Store','Date'))
# full_train['Week_In_Year']=full_train.Date.astype(np.datetime64).apply(lambda d: datetime.date.isocalendar(d)[1])
# full_train.Date=full_train.Date.astype(np.datetime64)
# full_train['IsHoliday']=full_train.IsHoliday_x.astype('int')
# # full_train['Store']=full_train.Store.astype('U')
# # full_train['Dept']=full_train.Dept.astype('U')
# full_train['Week_In_Year']=full_train.Week_In_Year.astype('U')
# full_train.drop(['IsHoliday_x','IsHoliday_y'],axis=1,inplace=True)
# depts1=full_train[full_train.Dept==1][['Store','Dept','Type','Date','Weekly_Sales','Size','Temperature','Fuel_Price','CPI','Unemployment','IsHoliday','Week_In_Year']]
# depts1_isHoliday=depts1.IsHoliday
# depts1_train=depts1[depts1.Date<np.datetime64('2012-02-05')]
# depts1_test=depts1[depts1.Date>=np.datetime64('2012-02-05')]
# for df in (depts1_train,depts1_test):
#     df.index=np.arange(len(df))
    


# full_pipeline=DataFramePipeline([
# #     FeaturePipeline('Store','',Pipeline([('onehot',LabelBinarizerEx(['Store']))])),
# #     FeaturePipeline('Dept','',Pipeline([('onehot',LabelBinarizerEx(['Dept']))])),
# #     FeaturePipeline('Type','',Pipeline([('onehot',LabelBinarizerEx(['Type']))])),
# #     FeaturePipeline('Size','Size',Pipeline([('scale',StandardScaler())])),
# #     FeaturePipeline('Temperature','Temperature',Pipeline([('scale',StandardScaler())])),
# #     FeaturePipeline('Fuel_Price','Fuel_Price',Pipeline([('scale',StandardScaler())])),
# #     FeaturePipeline('CPI','CPI',Pipeline([('scale',StandardScaler())])),
# #     FeaturePipeline('Unemployment','Unemployment',Pipeline([('scale',StandardScaler())])),
#     FeaturePipeline('Week_In_Year','',Pipeline([('onehot',LabelBinarizerEx(['Week_In_Year']))])),
# ]
# )

# depts1p_train=full_pipeline.fit_transform(depts1_train)
# depts1p_train.drop(['Store','Dept','Type','Week_In_Year','IsHoliday','Date'],axis=1,inplace=True)
# depts1p_target=depts1p_train.Weekly_Sales
# depts1p_train=depts1p_train.drop('Weekly_Sales',axis=1)
# depts1p_train.head()

In [ ]:
# lr=LinearRegression()
# gbt=GradientBoostingRegressor()
# lr.fit(depts1p_train,depts1p_target)
# sorted(zip(depts1_train.columns,lr.coef_),key=lambda x:x[1],reverse=True)


In [ ]:
# depts1p_test=full_pipeline.transform(depts1_test)
# depts1p_test_isHoliday=depts1p_test.IsHoliday
# depts1p_test.drop(['Store','Dept','Type','Week_In_Year','IsHoliday','Date'],axis=1,inplace=True)
# depts1p_sales=depts1p_test.Weekly_Sales
# depts1p_test=depts1p_test.drop('Weekly_Sales',axis=1)
# depts1p_test.head()

In [ ]:
def score(predict_data,validate_data,isHoliday):
    weights = np.array([5 if holiday else 1 for holiday in isHoliday])
    return np.sum(np.abs((predict_data-validate_data)*weights)) / np.sum(weights)

# score(lr.predict(depts1p_test),depts1p_sales,depts1p_test_isHoliday)
# len(lr.predict(depts1p_test)),len(depts1p_sales),len(depts1p_test_isHoliday)

In [ ]:
# gbt=GradientBoostingRegressor(n_estimators=100)
# gbt.fit(depts1p_train,depts1p_target)
# score(gbt.predict(depts1p_test),depts1p_sales,depts1p_test_isHoliday)

In [ ]:
# sorted(zip(depts1p_train.columns,gbt.feature_importances_),key=lambda x:x[1],reverse=True)

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC,LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBClassifier,XGBRegressor

base_clfs=[
      LinearRegression(n_jobs=-1),
    
      RandomForestRegressor(n_jobs=-1),
      GradientBoostingRegressor(),
    XGBRegressor(),
#       AdaBoostClassifier(), 
#       ExtraTreesClassifier(n_jobs=-1)
]
from sklearn.model_selection import cross_val_score

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *

def trainModels(train_data, target):
    scores=[cross_val_score(clf,train_data,target,scoring='neg_mean_absolute_error',cv=5,verbose=1).mean() for clf in base_clfs]

    labels=[c.__class__.__name__[:3] for c in base_clfs]
#     X=np.arange(len(base_clfs))
#     bar(X,scores,tick_label=labels,color='rgb')
#     ylim(0.5,1.0)
#     show()
    print(sorted(zip(labels,scores),key=lambda x:x[1],reverse=True))
    
# trainModels(depts1p_train, depts1p_target)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid_set=[
# #                 {'C':[0.01,0.1,0.5,1.]},
# #                 {'C':[1.,10.,],'kernel':['rbf','poly'],'gamma':[0.01,0.1,1.],'coef0':[1.,10.,]},
# #                 {'n_estimators':[50,100,200,300],'max_depth':[5,10,15]},
# #                 {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,200,300],'max_depth':[3,5,8]},
#     {'learning_rate':[0.1],'n_estimators':[1500,2000,3000],'max_depth':[5]},
# #                 {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,200,300]},
# #                 {'n_estimators':[50,100,200,300],'max_depth':[5,10,15]},
# #                 {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,200,300],'max_depth':[5,10,15],'gamma':[0.01,0.1,0.5]},
#                ]

# base_clfs=[GradientBoostingRegressor()]
# def tuneModels(train_data,target):
#     results=[]
#     for i in range(len(base_clfs)):
#         gs=GridSearchCV(estimator=base_clfs[i],param_grid=param_grid_set[i],scoring=score,verbose=1,cv=5)
#         gs.fit(train_data,target)
#         results.append((gs.best_estimator_,gs.best_score_))
#     print(sorted(results,key=lambda x:x[1],reverse=True))
#     return results

# tuneModels(depts1_train, depts1_target)

In [ ]:
train_data = pd.read_csv('raw_data/train.csv')
test_data = pd.read_csv('raw_data/test.csv')

all_df=[train_data,test_data]

for i in range(len(all_df)):
    df=all_df[i]
    all_df[i]=df.merge(stores,left_on='Store',right_on='Store').merge(features,left_on=('Store','Date'),right_on=('Store','Date'))
    df=all_df[i]
    df.Date=df.Date.astype(np.datetime64)
    df['Week_In_Year']=df.Date.apply(lambda d: datetime.date.isocalendar(d)[1])
    df['Year']=df.Date.apply(lambda d:d.year)
    
    df['IsHoliday']=df.IsHoliday_x.astype('int')
    df['Store']=df.Store.astype('U')
    df['Week_In_Year']=df.Week_In_Year.astype('U')
    df.drop(['IsHoliday_x','IsHoliday_y'],axis=1,inplace=True)
    
train_data=all_df[0]
test_data=all_df[1]

In [ ]:
# import time
# t=time.time()

# models={}
# depts=sorted(test_data.Dept.unique())
# for d in depts:
#     depts_train=train_data[train_data.Dept==d][['Store','Type','Weekly_Sales','Size',
#                                                 'Temperature','Fuel_Price','CPI','Unemployment','Week_In_Year']]
#     depts_train.index=np.arange(len(depts_train))
#     sales_train=depts_train.Weekly_Sales
    
#     pipeline=DataFramePipeline([
#         FeaturePipeline('Store','',Pipeline([('onehot',LabelBinarizerEx(['Store']))])),
#         FeaturePipeline('Type','',Pipeline([('onehot',LabelBinarizerEx(['Type']))])),
#         FeaturePipeline('Temperature','Temperature',Pipeline([('impute',Imputer(strategy='median'))])),
#         FeaturePipeline('Fuel_Price','Fuel_Price',Pipeline([('impute',Imputer(strategy='median'))])),
#         FeaturePipeline('CPI','CPI',Pipeline([('impute',Imputer(strategy='median'))])),
#         FeaturePipeline('Unemployment','Unemployment',Pipeline([('impute',Imputer(strategy='median'))])),
#         FeaturePipeline('Week_In_Year','',Pipeline([('onehot',LabelBinarizerEx(['Week_In_Year']))])),
#     ]
#     )
#     depts_p=pipeline.fit_transform(depts_train)
#     depts_p.drop(['Weekly_Sales','Store','Type','Week_In_Year'],axis=1,inplace=True)
    
#     gbt=GradientBoostingRegressor(n_estimators=1000)
#     gbt.fit(depts_p,sales_train)
#     models[d]=(pipeline, gbt)
    
# preds=[]
# store_depts=test_data.groupby(['Store','Dept']).count().index
# for sd in store_depts:
#     raw_test=test_data[np.logical_and(test_data.Store==sd[0],test_data.Dept==sd[1])]
#     depts_test=raw_test[['Store','Type','Size','Temperature','Fuel_Price','CPI','Unemployment','Week_In_Year']]
#     depts_test.index=np.arange(len(depts_test))

#     pipeline=models[sd[1]][0]
#     depts_p=pipeline.transform(depts_test)
#     depts_p.drop(['Store','Type','Week_In_Year'],axis=1,inplace=True)
#     sales_pred=models[sd[1]][1].predict(depts_p)
#     preds.append(pd.DataFrame({'Id':str(sd[0])+'_'+str(sd[1])+'_'+raw_test.Date.astype('U'),'Weekly_Sales':sales_pred.astype('int')}))
    
# result=pd.concat(preds)
# result.to_csv('output/regression/result.csv',index=False,header=True)

# 'total time %ds' % int(time.time()-t)

score 3235, rank 236

features excluding isHoliday, markdowns

score 3047

features add year, gbdt 1000

In [ ]:
test_data.describe()

In [ ]:
train_data.describe()

In [ ]:
onehot_pipeline=DataFramePipeline([
    FeaturePipeline('Store','',Pipeline([('onehot',LabelBinarizerEx(['Store']))])),
    FeaturePipeline('Type','',Pipeline([('onehot',LabelBinarizerEx(['Type']))])),
    FeaturePipeline('Week_In_Year','',Pipeline([('onehot',LabelBinarizerEx(['Week_In_Year']))])),
]
)
train_p = onehot_pipeline.fit_transform(train_data)
train_p.describe()



In [ ]:
md_cols=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']
hasMarkDown=np.array([False]*len(train_p))
for c in md_cols:
    hasMarkDown=np.logical_or(np.logical_not(np.isnan(train_p[c])),hasMarkDown)
    
train_md=train_p[hasMarkDown]
train_md=train_md.fillna(0)

In [ ]:
import time
t=time.time()

models_md={}
models={}
depts=sorted(test_data.Dept.unique())
for d in depts:
    print('dept %d'%d)
    sales_train=train_p[train_p.Dept==d].Weekly_Sales
    depts_train=train_p[train_p.Dept==d].drop(['Store','Type','Weekly_Sales','Week_In_Year','Date','IsHoliday']+md_cols,axis=1)
    depts_train.index=np.arange(len(depts_train))
    
    gbt=GradientBoostingRegressor(n_estimators=1000)
    gbt.fit(depts_train,sales_train)
    models[d]=gbt
    
#     sales_train=train_md[train_md.Dept==d].Weekly_Sales
#     depts_train=train_md[train_md.Dept==d].drop(['Store','Type','Weekly_Sales','Week_In_Year','Date','IsHoliday'],axis=1)
#     depts_train.index=np.arange(len(depts_train))

#     gbt=RandomForestRegressor(n_estimators=300,max_depth=10)
#     gbt.fit(depts_train,sales_train)
#     models_md[d]=gbt
    

'total time %ds' % int(time.time()-t)

In [ ]:
t=time.time()

test_p=onehot_pipeline.transform(test_data)
test_pipelines={}
stores=test_p.Store.unique()
for st in stores:
    pipeline=DataFramePipeline([
        FeaturePipeline('CPI','CPI',Pipeline([('impute',Imputer(strategy='median'))])),
        FeaturePipeline('Unemployment','Unemployment',Pipeline([('impute',Imputer(strategy='median'))])),
    ]
    )
    pipeline.fit(test_p[test_p.Store==st])
    test_pipelines[st]=pipeline
    
preds=[]
store_depts=test_p.groupby(['Store','Dept']).count().index
md_pred=0

for sd in store_depts:
    depts_test=test_p[np.logical_and(test_p.Store==sd[0],test_p.Dept==sd[1])]
    depts_test.index=np.arange(len(depts_test))
    
    
    depts_p=test_pipelines[sd[0]].transform(depts_test)
    
#     hasMarkDown=np.array([False]*len(depts_p))
#     for c in md_cols:
#         hasMarkDown=np.logical_or(np.logical_not(np.isnan(depts_p[c])),hasMarkDown)

#     test_md=depts_p[hasMarkDown]
#     if len(test_md)>0:
#         dates=test_md.Date
#         test_md=test_md.fillna(0)
#         test_md=test_md.drop(['Store','Type','Week_In_Year','Date','IsHoliday'],axis=1)
#         sales_pred=models_md[sd[1]].predict(test_md)
#         preds.append(pd.DataFrame({'Id':str(sd[0])+'_'+str(sd[1])+'_'+dates.astype('U'),'Weekly_Sales':sales_pred.astype('int')}))
#         md_pred+=len(sales_pred)
    
#     test_no_md=depts_p[np.logical_not(hasMarkDown)]
#     if len(test_no_md)>0:
    dates=depts_p.Date
    test_no_md=depts_p.drop(['Store','Type','Week_In_Year','Date','IsHoliday']+md_cols,axis=1)
    sales_pred=models[sd[1]].predict(test_no_md)
    preds.append(pd.DataFrame({'Id':str(sd[0])+'_'+str(sd[1])+'_'+dates.astype('U'),'Weekly_Sales':sales_pred.astype('int')}))
    
    
result=pd.concat(preds)
result.to_csv('output/regression/result.csv',index=False,header=True)

# print(md_pred, len(test_p)-md_pred)
'total time %ds' % int(time.time()-t)

In [ ]:
test_p.describe()

In [ ]:
sales1=train_md[train_md.Dept==1].Weekly_Sales
train1=train_md[train_md.Dept==1].drop(['Store','Type','Weekly_Sales','Week_In_Year','Date','IsHoliday'],axis=1)
gbt=GradientBoostingRegressor()
gbt.fit(train1,sales1)
sorted(zip(train1.columns,gbt.feature_importances_),key=lambda x:x[1],reverse=True)